In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import keras as ks
from keras.preprocessing import image
from keras.applications import imagenet_utils
from IPython.display import Image
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
weights = ks.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet', pooling='avg')

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
def load_images_from_folder(f, l, m):
    e_l = []
    feat_l = []
    for filename in tqdm(os.listdir(f)):
        img_path = os.path.join(f, filename)
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        #feat = m.predict(x)

        e_l.append(x)
        feat_l.append(l)
        
    return e_l, feat_l

In [4]:
def load_image(f):
    img_path = os.path.join('', f)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
        
    return x

In [5]:
enfermos, feat_enfermos = load_images_from_folder('fondo_de_ojo_train/enfermo/', 0, weights)

100%|██████████| 827/827 [00:07<00:00, 105.75it/s]


In [6]:
sanos, feat_sanos = load_images_from_folder('fondo_de_ojo_train/sano/', 1, weights)

100%|██████████| 815/815 [00:08<00:00, 98.24it/s] 


In [7]:
X = sanos + enfermos
Y = feat_sanos + feat_enfermos
# suffle
#np.random.shuffle(data)
#data = np.array(data)
#X = data[: , 0]
#Y = data[:, 1]
X = np.array(X)
Y = np.array(Y)

In [8]:
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

Training data shape :  (1313, 224, 224, 3) (1313,)
Testing data shape :  (329, 224, 224, 3) (329,)


In [9]:
train_X = train_X / 255.
test_X = test_X / 255.

In [10]:
classes = np.unique(train_Y)
nClasses = len(classes)
nClasses

2

In [11]:
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
 
# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

Original label: 1
After conversion to one-hot: [0. 1.]


In [12]:
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y_one_hot, test_size=0.2)

In [13]:
train_X

array([[[[-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         ...,
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854]],

        [[-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         ...,
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854]],

        [[-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         ...,
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854],
         [-0.00379854, -0.00379854, -0.00379854]],

        ...,

        [[-0.00379854, -0.00379854, -0

In [14]:
INIT_LR = 1e-3
epochs = 150
batch_size = 64

mobile = Sequential()
mobile.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(224,224,3)))
mobile.add(LeakyReLU(alpha=0.1))
mobile.add(MaxPooling2D((2, 2),padding='same'))
mobile.add(Dropout(0.5))

mobile.add(Flatten())
mobile.add(Dense(32, activation='linear'))
mobile.add(LeakyReLU(alpha=0.1))
mobile.add(Dropout(0.5)) 
mobile.add(Dense(nClasses, activation='softmax'))

mobile.summary()

mobile.compile(loss=ks.losses.categorical_crossentropy, optimizer=ks.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                12845088  
___________________________

In [ ]:
mobile_train_dropout = mobile.fit(train_X, train_Y, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_Y))

Instructions for updating:
Use tf.cast instead.
Train on 1050 samples, validate on 263 samples
Epoch 1/150
1050/1050 [==============================] - 35s 33ms/step - loss: 0.6997 - acc: 0.5171 - val_loss: 0.6933 - val_acc: 0.5399
Epoch 2/150
1050/1050 [==============================] - 33s 32ms/step - loss: 0.6907 - acc: 0.5190 - val_loss: 0.6903 - val_acc: 0.5361
Epoch 3/150
1050/1050 [==============================] - 36s 35ms/step - loss: 0.6899 - acc: 0.5267 - val_loss: 0.7038 - val_acc: 0.4715
Epoch 4/150
1050/1050 [==============================] - 38s 36ms/step - loss: 0.6883 - acc: 0.5238 - val_loss: 0.6894 - val_acc: 0.5361
Epoch 5/150
1050/1050 [==============================] - 38s 36ms/step - loss: 0.6872 - acc: 0.5524 - val_loss: 0.7064 - val_acc: 0.4981
Epoch 6/150
1050/1050 [==============================] - 39s 37ms/step - loss: 0.6863 - acc: 0.5600 - val_loss: 0.6895 - val_acc: 0.5133
Epoch 7/150
1050/1050 [==============================] - 36s 34ms/step - loss: 0.67

In [159]:
mobile_eval = mobile.evaluate(test_X, test_Y_one_hot, verbose=1)
 
print('Test loss:', mobile_eval[0])
print('Test accuracy:', mobile_eval[1])

329/329 [==============================] - 3s 10ms/step
Test loss: 0.6929858323288544
Test accuracy: 0.510638297962925


In [130]:
# calculate predictions
predictions = mobile.predict(test_X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [151]:
enfermito = load_image('fondo_de_ojo_train/enfermo/train_enfermo1.jpg')
sanito = load_image('fondo_de_ojo_train/sano/train_sano1.jpg')

In [152]:
n1 = enfermito/255
n2 = sanito/255
n = np.array([n1, n2])
n.shape

(2, 224, 224, 3)

In [153]:
predictions = mobile.predict(n)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

[1.0, 1.0]
